# Генерация данных #

### Загрузка пакетов ###

In [354]:
#обработка музыки
import librosa

# загрузка mp3 файлов
import audioread.ffdec  # Use ffmpeg decoder
import ffmpeg

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (20,10)

#файлы в директории
from glob import glob

#воспроизведение музыки
from IPython.display import Audio

import json

### Первичная загрузка данных ###

In [358]:
# путь до директориис музыкой
music_dir = './music'

#имена всех музыкальных файлов
musicfiles = glob(music_dir+ "\*.mp3")

In [247]:
#загрузка файла
aro = audioread.ffdec.FFmpegAudioFile(musicfiles[1])
y, sr = librosa.load(aro)

In [248]:
#весёлая нарезка детей
#количество элементов для 7.5 секунд
k = librosa.time_to_samples(7.5, sr=sr)
music_fragments=[]
#нарезка
for i in range(int(np.ceil(y.shape[0] / k))):
    music_fragments.append(y[i * k: (i+1) * k])
#приведение последнего объекта к 7.5 секундам
music_fragments[-1] = np.append(music_fragments[-1], np.zeros((k- music_fragments[-1].shape)))
music_fragments = np.array(music_fragments)

In [249]:
#вызов музыки
Audio(data= music_fragments[0], rate=sr)

In [250]:
#добавление шума
STD_n= 0.01
noise=np.random.normal(0, STD_n, k)

music_noise = music_fragments + noise

In [252]:
#вызов музыки c шумом
Audio(data= music_noise[0],  rate=sr)

In [257]:
import soundfile as sf

In [266]:
save_dir = './data/'
# сохранение музыки
sf.write(save_dir+'exampleee.wav', music_noise[0], sr)

### Рефакторинг ###

In [452]:
#класс генератора данных 
class DataGenerator:
    def __init__(self, audio_path):
        """
        Инициализация объекта класса DataGenerator
        
        Параметры:
            audio_path (str) - путь к аудиофайлу
            
        Возвращает:
            None
            
        Примечание:
            self.path - путь к файлу
            self.y - звуковой временной ряд
            self.t - частота дискретизации y
            self.samples - нарезанные музыкальные фрагменты
            self.number_of_samples - количество нарезанных фрагментов
        """
        
        self.name = audio_path
        self.y = None
        self.sr = None
        self.samples = None
        self.number_of_samples = None
        self.noised_samples = None
        self.noise_level = None
        
        self.read_audio(audio_path)
        
        
    def read_audio(self, audio_path):
        """
        Чтение аудиофайла, лежащего в audio_path
        
        Параметры:
            audio_path (str) - путь к аудиофайлу
            
        Возвращает:
            None
            
        Примечание:
            Инициализирует атрибуты класса:
                y (np.ndarray) - звуковой временной ряд
                sr (int > 0) - частота дискретизации y
        """
        self.y, self.sr = librosa.load(audioread.ffdec.FFmpegAudioFile(audio_path))
        
        
    def to_samples(self, t):
        """
        Нарезает звуковой временной ряд self.y на дорожки длиной t секунд
        
        Параметры:
            t (number or np.ndarray) - длина нарезки аудиофрагментов
            
        Возвращает:
            None
            
        Примечание:
            Инициализирует атрибут класса:
                self.music_fragments (np.ndarray) - массив, содержащий звуковые временные
                    ряды длины t
            
            Нарезанные фрагменты не перекрывают друг друга, за исключением, может быть, последнего,
                который подгоняется под длину t
        """
        self.number_of_samples = librosa.time_to_samples(t, sr = self.sr)
        _music_fragments = []
        
        #нарезка
        for i in range(int(np.ceil(self.y.shape[0] / self.number_of_samples))):
            _music_fragments.append(self.y[i * self.number_of_samples: (i + 1) * self.number_of_samples])
            
        #приведение последнего объекта к 7.5 секундам
        _music_fragments[-1] = np.append(_music_fragments[-1], np.zeros((self.number_of_samples - _music_fragments[-1].shape)))
        
        self.samples = np.array(_music_fragments)
        
    
    def get_samples(self):
        """
        Выгружает нарезанные фрагменты длины t
        
        Параметры:
            None
            
        Возвращает:
            self.samples (np.ndarray) - массив, содержащий звуковые временные ряды длины t
            
        Примечание:
            Использует метод to_samples
        """
        return self.samples
   

    def add_noise(self, noise_level):
        """
        Добавляет шум к нарезанным музыкальным фрагментам samples
            в степени noise_level
            
        Параметры:
            samples (np.ndarray) - массив, содержащий звуковые временные ряды
            noise_level (number) - степень шума. от 0 до 1
            
        Возвращает:
            samples + noise (np.ndarray) - нарезанные музыкальные фрагменты с добавленным шумом
            
        Примечание:
        """
        self.noise_level = noise_level
        self.noised_samples = self.samples + np.random.normal(0, noise_level, self.number_of_samples)
        
        
    def save_samples(self, save_dir):
        """
        Сохраняет нарезанные музыкальные фрагменты samples
            в директории save_dir
            
        Параметры:
            save_dir (str) - директория, куда сохраняются файлы
            
        Возвращает:
            None
            
        Примечание:
            Сохраненные файлы называются по следующему принципу:
                имя-исходного-файла_номер-фрагмента.wav
        """
        for i in range(len(self.samples)):
            sf.write(save_dir + self.name[8:-4] + "_" + str(i) + ".wav", self.samples[i], self.sr)
           
        
    def save_noised_samples(self, save_dir):
        """
        Сохраняет нарезанные музыкальные фрагменты samples с добавленным шумом в степени noise_level
            в директории save_dir
            
        Параметры:
            save_dir (str) - директория, куда сохраняются файлы
            noise_level (number) - степень шума. от 0 до 1
            
        Возвращает:
            None
            
        Примечание:
            Сохраненные файлы называются по следующему принципу:
                noised_степень-шума_имя-исходного-файла_номер-фрагмента.wav
        """
        
        for i in range(len(self.noised_samples)):
            sf.write(save_dir + "noised_" + str(noise_level) + "_"+ self.name[8:-4] + "_" 
                     + str(i) + ".wav", self.noised_samples[i], self.sr)
            
            
    def save_to_json(self, save_dir):
        """
        Сохраняет нарезанные музыкальные фрагменты samples
            в json файле save_dir
            
        Параметры:
            save_dir (str) - json файл, куда сохраняются данные
            
        Возвращает:
            None
            
        Примечание:
            Сохраненные файлы называются по следующему принципу:
                [{
                    init_dict:
                        {
                        name: str
                        sr: int > 0
                        noised: bool
                        noise_level: number
                        },
                    samples: list of lists of numbers
                 } , ...]
        """
        with open(save_dir, "r+") as file:
            _data = json.load(file)
            _data.append({'init_dict': {'name': self.name[8:], 'sr': self.sr, 'noised': 0, 'noise_level': None}, 
                          'samples': [list(_) for _ in self.samples]})
            file.seek(0)
            json.dump(_data, file)
            
    def save_noised_to_json(self, save_dir):
        """
        Сохраняет нарезанные музыкальные фрагменты samples
            в json файле save_dir
            
        Параметры:
            save_dir (str) - json файл, куда сохраняются данные
            
        Возвращает:
            None
            
        Примечание:
            Сохраненные файлы называются по следующему принципу:
                [{
                    init_dict:
                        {
                        name: str
                        sr: int > 0
                        noised: bool
                        noise_level: number
                        },
                    samples: list of lists of numbers
                 } , ...]
        """
        with open(save_dir, "r+") as file:
            _data = json.load(file)
            _data.append({'init_dict': {'name': self.name[8:], 'sr': self.sr, 'noised': 1, "noise_level": self.noise_level}, 
                          'samples': [list(_) for _ in self.samples]})
            file.seek(0)
            json.dump(_data, file)

In [440]:
ex = DataGenerator(musicfiles[1])

In [441]:
ex.to_samples(7.5)

In [442]:
for i in np.arange(0.01, 0.1, 0.01):
    ex.save_noised_samples('./wav_data/', round(i, 2))

In [340]:
Audio(data = ex.samples[10],  rate=ex.sr)